In [120]:
import pandas as pd
X = pd.read_csv('~/data/train_input_Z61KlZo.csv') 
X = X.drop(columns=["ID"])


/tmp/ipykernel_15638/1957515299.py:2: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X = pd.read_csv('~/data/train_input_Z61KlZo.csv')


In [123]:
top_20_features = [
    "ANCIENNETE", "TAILLE1", "DEROG13", "ACTIVIT2", "KAPITAL17", "SURFACE1", 
    "KAPITAL22", "ANNEE_ASSURANCE", "FRCH2", "NBBAT9", "SURFACE11", "EQUIPEMENT6", 
    "SURFACE17", "FRCH1", "PROPORTION_31", "NBSINSTRT", "KAPITAL21", "KAPITAL18", 
    "NBBAT14", "NBSINCONJ"
]

X = X[top_20_features]
print(X.head())


   ANCIENNETE           TAILLE1 DEROG13 ACTIVIT2  KAPITAL17  SURFACE1  \
0           0  05 - [1M - 1.5M]    -999     ACT1        0.0      1600   
1           0  02 - [250k-500k]    -999     ACT1        0.0       625   
2           2  01 - [0   -250k]    -999     ACT1        0.0       225   
3           0  01 - [0   -250k]    -999     ACT1        0.0       100   
4           1  01 - [0   -250k]    -999     ACT1        0.0       175   

   KAPITAL22  ANNEE_ASSURANCE FRCH2  NBBAT9  SURFACE11  EQUIPEMENT6  \
0        0.0         1.000000  -999       3        0.0            9   
1        0.0         1.000000  -999       0        0.0            9   
2        0.0         0.402740  -999       0        0.0            8   
3        0.0         0.246575  -999       1        0.0            2   
4        0.0         0.838356  -999       1        0.0            7   

   SURFACE17 FRCH1 PROPORTION_31  NBSINSTRT  KAPITAL21  KAPITAL18  NBBAT14  \
0          0     2     06. <= 60       0.00     2000.0  

In [122]:
# Traitement des valeurs manquantes dans les colonnes numériques
print("Traitement des valeurs manquantes dans les colonnes numériques...")
numeric_columns = X.select_dtypes(include=['number']).columns

# Remplir les NaN avec 0 pour les colonnes numériques
X[numeric_columns] = X[numeric_columns].fillna(0)

# Identifier les colonnes non numériques
fill_cols = [item for item in X.columns if item not in numeric_columns]

# Remplir les NaN des colonnes non numériques avec une valeur par défaut (-999)
X[fill_cols] = X[fill_cols].fillna(-999)

print("Traitement des valeurs manquantes terminé.")

Traitement des valeurs manquantes dans les colonnes numériques...
Traitement des valeurs manquantes terminé.


In [124]:
from category_encoders import CountEncoder
# Préparation des données pour l'entraînement
print("Préparation des données pour l'entraînement...")

# Encodage des variables catégoriques avec CountEncoder
encoder = CountEncoder(cols=fill_cols)
encoder.fit(X)
X = encoder.transform(X)

print("Préparation terminée.")

Préparation des données pour l'entraînement...
Préparation terminée.


In [97]:
Y = pd.read_csv('~/data/train_output_DzPxaPY.csv')

In [98]:
Y.head()

,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


In [99]:
y_freq = Y["FREQ"]
y_CM = Y["CM"]

In [100]:
for col in X.select_dtypes(include=["object", "category"]).columns:
    X[col] = X[col].astype("category").cat.codes

In [102]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.02, random_state=102)
y_train_freq = Y_train["FREQ"]
y_test_freq = Y_test["FREQ"]

y_train_CM = Y_train["CM"]
y_test_CM = Y_test["CM"]

model_freq = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=10)

model_freq.set_params(
    eval_metric="rmse",
    learning_rate=0.1,
    max_depth=5
)

model_freq.fit(
    X_train, y_train_freq,
    eval_set=[(X_train, y_train_freq), (X_test, y_test_freq)],  
    verbose=True  
)



[0]	validation_0-rmse:0.34658	validation_1-rmse:0.25630
[1]	validation_0-rmse:0.33510	validation_1-rmse:0.25627
[2]	validation_0-rmse:0.32440	validation_1-rmse:0.25620
[3]	validation_0-rmse:0.31443	validation_1-rmse:0.25616
[4]	validation_0-rmse:0.30515	validation_1-rmse:0.25613
[5]	validation_0-rmse:0.29653	validation_1-rmse:0.25611
[6]	validation_0-rmse:0.28852	validation_1-rmse:0.25607
[7]	validation_0-rmse:0.28110	validation_1-rmse:0.25604
[8]	validation_0-rmse:0.27423	validation_1-rmse:0.25598
[9]	validation_0-rmse:0.26785	validation_1-rmse:0.25597


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [103]:
import xgboost as xgb

model_CM = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=5)

model_CM.set_params(
    eval_metric="rmse",
    learning_rate=0.05,
    max_depth=5
)

model_CM.fit(
    X_train, y_train_CM,
    eval_set=[(X_train, y_train_CM), (X_test, y_test_CM)],  
    verbose=True  
)


[0]	validation_0-rmse:6700.09246	validation_1-rmse:6362.98646
[1]	validation_0-rmse:6693.94913	validation_1-rmse:6363.24305
[2]	validation_0-rmse:6688.26499	validation_1-rmse:6363.29316
[3]	validation_0-rmse:6681.84824	validation_1-rmse:6363.75377
[4]	validation_0-rmse:6676.13860	validation_1-rmse:6364.36288


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [104]:
# Obtenir l'importance des variables
importance = model_CM.get_booster().get_score(importance_type='weight')

importance_df = pd.DataFrame(list(importance.items()), columns=["Variable", "Importance"])
importance_df = importance_df.sort_values(by="Importance", ascending=False)

print(importance_df.head(20))

           Variable  Importance
1           TAILLE1        18.0
11      EQUIPEMENT6        13.0
5          SURFACE1        13.0
12        SURFACE17        11.0
14    PROPORTION_31        11.0
7   ANNEE_ASSURANCE        10.0
0        ANCIENNETE         8.0
17          NBBAT14         8.0
8             FRCH2         7.0
10        SURFACE11         6.0
2           DEROG13         5.0
18        NBSINCONJ         5.0
3          ACTIVIT2         5.0
6         KAPITAL22         5.0
4         KAPITAL17         4.0
13            FRCH1         4.0
9            NBBAT9         3.0
16        KAPITAL21         3.0
15        NBSINSTRT         1.0


In [105]:
y_pred_freq = model_freq.predict(X_test)
rmse = root_mean_squared_error(y_test_freq, y_pred_freq)
print(f"FREQ : root mean Squared Error: {rmse}")

y_pred_CM = model_CM.predict(X_test)
rmse = root_mean_squared_error(y_test_CM, y_pred_CM)
print(f"CM : Root mean Squared Error: {rmse}")

y_pred_charge = y_pred_freq * y_pred_CM * Y_test["ANNEE_ASSURANCE"]
rmse = root_mean_squared_error(Y_test["CHARGE"], y_pred_charge)
print(f"Charge : Root mean Squared Error: {rmse}")

FREQ : root mean Squared Error: 0.2559711181832736
CM : Root mean Squared Error: 6364.3629579367625
Charge : Root mean Squared Error: 6368.423217429236


In [106]:
y_pred_charge

229520    0.353282
161202    1.201693
173372    1.021837
161374    6.132223
200578    1.916062
            ...   
234913    2.079870
111392    0.120706
263539    0.032850
282028    4.618061
230767    0.233502
Name: ANNEE_ASSURANCE, Length: 7673, dtype: float64

In [113]:
X_submission = pd.read_csv("~/data/test_input_5qJzHrr.csv")

/tmp/ipykernel_15638/2995366613.py:1: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_submission = pd.read_csv("~/data/test_input_5qJzHrr.csv")


In [127]:
# Traitement des données de test
print("Traitement des données de test...")

# Remplir les valeurs manquantes
X_submission[numeric_columns] = X_submission[numeric_columns].fillna(0)
X_submission[fill_cols] = X_submission[fill_cols].fillna(-999)

# Suppression des colonnes inutiles
X_test_model = X_submission[top_20_features]

# Transformation des données de test avec le même encoder
X_test_enc = encoder.transform(X_test_model)

print("Traitement des données de test terminé.")

Traitement des données de test...
Traitement des données de test terminé.


In [131]:
# Prédictions sur l'ensemble de test
print("Prédictions sur l'ensemble de test...")

# Prédire 'FREQ' et 'CM' sur les données de test
y_pred_freq = model_freq.predict(X_test_enc)
y_pred_cm = model_CM.predict(X_test_enc)

Prédictions sur l'ensemble de test...


In [134]:
# Combiner les prédictions
y_pred = pd.concat([
    X_submission[['ID', 'ANNEE_ASSURANCE']].reset_index(drop=True),
    pd.DataFrame(y_pred_freq, columns=['FREQ']),
    pd.DataFrame(y_pred_cm, columns=['CM'])
], axis=1)

In [135]:
# Calculer la colonne 'CHARGE' en multipliant 'FREQ', 'CM' et 'ANNEE_ASSURANCE'
y_pred['CHARGE'] = y_pred['CM'] * y_pred['FREQ'] * y_pred['ANNEE_ASSURANCE']

print("Prédictions terminées.")

Prédictions terminées.


In [136]:
# Exporter les prédictions dans un fichier CSV
print("Exportation des résultats...")
y_pred.to_csv('submission.csv', index=False)
print("Fichier de soumission créé : 'submission.csv'")

Exportation des résultats...
Fichier de soumission créé : 'submission.csv'
